# Sherlock Holmes RNN 

In this project, I will implement a Recurrent Neural Network (RNN) architecture to create an English language sequence generator capable of building semi-coherent English sentences from scratch by building them up character-by-character.  For the training set, I will be using a complete version of Sir Arthur Conan Doyle's classic book The Adventures of Sherlock Holmes.

## Step 1:  Preprocessing a text dataset

Our first task is to load in our large text corpus dataset for use in training, and on it we perform several light pre-processing tasks.

In [1]:
# read in the text, transforming everything to lower case
text = open('data/holmes.txt').read().lower()
print('our original text has ' + str(len(text)) + ' characters')

our original text has 581864 characters


Next, lets examine a bit of the raw text.  Because we are interested in creating sentences of English words automatically by building up each word character-by-character, we only want to train on valid English words.  In other words - we need to remove all of the other junk characters that aren't words!

In [2]:
### print out the first 1000 characters of the raw text to get a sense of what we need to throw out
text[:2000]

"\ufeffproject gutenberg's the adventures of sherlock holmes, by arthur conan doyle\n\nthis ebook is for the use of anyone anywhere at no cost and with\nalmost no restrictions whatsoever.  you may copy it, give it away or\nre-use it under the terms of the project gutenberg license included\nwith this ebook or online at www.gutenberg.net\n\n\ntitle: the adventures of sherlock holmes\n\nauthor: arthur conan doyle\n\nposting date: april 18, 2011 [ebook #1661]\nfirst posted: november 29, 2002\n\nlanguage: english\n\n\n*** start of this project gutenberg ebook the adventures of sherlock holmes ***\n\n\n\n\nproduced by an anonymous project gutenberg volunteer and jose menendez\n\n\n\n\n\n\n\n\n\nthe adventures of sherlock holmes\n\nby\n\nsir arthur conan doyle\n\n\n\n   i. a scandal in bohemia\n  ii. the red-headed league\n iii. a case of identity\n  iv. the boscombe valley mystery\n   v. the five orange pips\n  vi. the man with the twisted lip\n vii. the adventure of the blue carbuncle\nvii

Wow - there's a lot of junk here (i.e., weird uncommon character combinations - as this first character chunk contains the title and author page, as well as table of contents)!  e.g., all the carriage return and newline sequences '\n' and '\r' sequences.  We want to train our RNN on a large chunk of real english sentences - we don't want it to start thinking non-english words or strange characters are valid! - so lets clean up the data a bit.

First, since the dataset is so large and the first few hundred characters contain a lot of junk, lets cut it out.  Lets also find-and-replace those newline tags with empty spaces.

In [3]:
### find and replace '\n' and '\r' symbols - replacing them 
text = text[1302:]
text = text.replace('\n',' ')    # replacing '\n' with '' simply removes the sequence
text = text.replace('\r',' ')

Lets see how the first 1000 characters of our text looks now!

In [4]:
### print out the first 1000 characters of the raw text to get a sense of what we need to throw out
text[:1000]

"is eyes she eclipses and predominates the whole of her sex. it was not that he felt any emotion akin to love for irene adler. all emotions, and that one particularly, were abhorrent to his cold, precise but admirably balanced mind. he was, i take it, the most perfect reasoning and observing machine that the world has seen, but as a lover he would have placed himself in a false position. he never spoke of the softer passions, save with a gibe and a sneer. they were admirable things for the observer--excellent for drawing the veil from men's motives and actions. but for the trained reasoner to admit such intrusions into his own delicate and finely adjusted temperament was to introduce a distracting factor which might throw a doubt upon all his mental results. grit in a sensitive instrument, or a crack in one of his own high-power lenses, would not be more disturbing than a strong emotion in a nature such as his. and yet there was but one woman to him, and that woman was the late irene a

Lets make sure we haven't left any other non-English/proper punctuation (commas, periods, etc., are ok) characters lurking around in the depths of the text.  You can do this by ennumerating all the text's unique characters, examining them, and then replacing any unwanted (non-english) characters with empty spaces!  Once we find all of the text's unique characters, we can remove all of the non-English/proper punctuation ones in the next cell.  Note: don't remove necessary punctuation marks!  (given in the cell below).  

In [5]:
### list all unique characters in the text and remove any non-english ones
# find all unique characters in the text

# remove non-english characters and character sequences
#punctuation = [' ', '!', ',', '.', ':', ';', '?']

text = text.replace('é',' ')
text = text.replace('è',' ')
text = text.replace('@',' ')
text = text.replace('â',' ')
text = text.replace('à',' ')
text = text.replace('"',' ')
text = text.replace('$',' ')
text = text.replace('%',' ')
text = text.replace('&',' ')
text = text.replace('(',' ')
text = text.replace(')',' ')
text = text.replace('*',' ')
text = text.replace('-',' ')
text = text.replace('/',' ')
text = text.replace(':',' ')
text = text.replace(';',' ')
text = text.replace('0',' ')
text = text.replace('1',' ')
text = text.replace('2',' ')
text = text.replace('3',' ')
text = text.replace('4',' ')
text = text.replace('5',' ')
text = text.replace('6',' ')
text = text.replace('7',' ')
text = text.replace('8',' ')
text = text.replace('9',' ')
    
# shorten any extra dead space created above
text = text.replace('  ',' ')

characters = (list(set(text)))
print(characters)

['n', ' ', 'a', 'd', 'm', 'z', 'j', '?', 'l', 's', ',', 'k', 'g', 'x', 'f', 'u', 'p', 'i', 't', "'", 'v', 'q', 'r', 'e', 'w', 'h', 'c', 'o', '.', 'b', 'y', '!']


Now that we have thrown out a good number of non-English characters/character sequences lets print out some statistics about the dataset - including number of total characters and number of unique characters.

In [6]:
# count the number of unique characters in the text
chars = sorted(list(set(text)))

# print some of the text, as well as statistics
print ("this corpus has " +  str(len(text)) + " total number of characters")
print ("this corpus has " +  str(len(chars)) + " unique characters")

this corpus has 574219 total number of characters
this corpus has 32 unique characters
